# Data processing

In [ ]:
import pandas as pd
import os
import re
from datasets import Dataset, DatasetDict



# Load CSV files
data_path = r"C:\Users\fabli\OneDrive\Desktop\fabliha\Masters Studies\AI in Healthcare\Assignments\high_risk_project\data"

patients_df = pd.read_csv(os.path.join(data_path, "patients.csv"))
conditions_df = pd.read_csv(os.path.join(data_path, "conditions.csv"))
observations_df = pd.read_csv(os.path.join(data_path, "observations.csv"))
medications_df = pd.read_csv(os.path.join(data_path, "medications.csv"))
immunizations_df = pd.read_csv(os.path.join(data_path, "immunizations.csv"))
encounters_df = pd.read_csv(os.path.join(data_path, "encounters.csv"))

# Find COVID-confirmed patients
covid_conditions = conditions_df[conditions_df['DESCRIPTION'].str.contains('COVID', case=False, na=False)]
covid_patients = covid_conditions['PATIENT'].unique()

# Find COVID recovery dates
covid_conditions['START'] = pd.to_datetime(covid_conditions['START'])
covid_conditions['STOP'] = pd.to_datetime(covid_conditions['STOP'], errors='coerce')
covid_conditions['RECOVERY_DATE'] = covid_conditions['STOP'].fillna(covid_conditions['START'] + pd.Timedelta(days=14))
covid_recovery_dates = covid_conditions[['PATIENT', 'RECOVERY_DATE']]

# Preparing observations
observations_df['DATE'] = pd.to_datetime(observations_df['DATE'])

# Only keeping observations of confirmed COVID patients
observations_covid_patients = observations_df[observations_df['PATIENT'].isin(covid_patients)]

# Merge observations with COVID recovery dates
merged_obs = pd.merge(observations_covid_patients, covid_recovery_dates, on='PATIENT', how='inner')

# Filter observations after recovery
post_recovery_obs = merged_obs[merged_obs['DATE'] > merged_obs['RECOVERY_DATE']]

# Calculate days after recovery
post_recovery_obs['days_after_recovery'] = (post_recovery_obs['DATE'] - post_recovery_obs['RECOVERY_DATE']).dt.days

# Keeping only symptom related observations (positive filtering), more symptoms can be added here, i am keeping it simple now
symptom_keywords = [
    'pain', 'fatigue', 'shortness of breath', 'cough', 'fever', 'headache', 
    'dizziness', 'nausea', 'diarrhea', 'sore throat', 'loss of taste', 'loss of smell'
]
symptoms_after_recovery = post_recovery_obs[post_recovery_obs['DESCRIPTION'].str.lower().str.contains('|'.join(symptom_keywords))]

''' Added post_covid_symptom label if they are likely related to covid or not , taking 60 as cutoff days, 
    by looking at the data distribution I prefered 60, but it can be changed for better data availability and actual medical preference.
    It would be better if we can use more post covid data.
'''
def label_post_covid(row):
    if row['days_after_recovery'] <= 60:  # i am keeping the value 60, but for improvement i would prefer it as a variable
        return 'likely related'
    else:
        return 'unlikely related'

symptoms_after_recovery['post_covid_symptom'] = symptoms_after_recovery.apply(label_post_covid, axis=1)

#  Build initial final_symptoms table
final_symptoms = symptoms_after_recovery[['PATIENT', 'DESCRIPTION', 'DATE', 'RECOVERY_DATE', 'days_after_recovery', 'VALUE', 'UNITS', 'post_covid_symptom']]

# Add patient info (Age, Gender, Race, Ethnicity)
patients_df['BIRTHDATE'] = pd.to_datetime(patients_df['BIRTHDATE'])
patients_df['AGE'] = (pd.to_datetime('2025-01-01') - patients_df['BIRTHDATE']).dt.days // 365

patient_info = patients_df[['Id', 'GENDER', 'RACE', 'ETHNICITY', 'AGE']].rename(columns={'Id': 'PATIENT'})
final_symptoms = final_symptoms.merge(patient_info, on='PATIENT', how='left')

# Add comorbidities (Other Diagnosed Conditions)
comorbidities = conditions_df[~conditions_df['DESCRIPTION'].str.contains('COVID', case=False, na=False)]
comorbidities_grouped = comorbidities.groupby('PATIENT')['DESCRIPTION'].apply(lambda x: ', '.join(x)).reset_index().rename(columns={'DESCRIPTION': 'COMORBIDITIES'})
final_symptoms = final_symptoms.merge(comorbidities_grouped, on='PATIENT', how='left')

# Add medications and immunizations info (for only patients who had symptoms)
symptom_patients = final_symptoms['PATIENT'].unique()

medications_covid_patients = medications_df[medications_df['PATIENT'].isin(symptom_patients)]
immunizations_covid_patients = immunizations_df[immunizations_df['PATIENT'].isin(symptom_patients)]

medications_grouped = medications_covid_patients.groupby('PATIENT')['DESCRIPTION'].apply(lambda x: ', '.join(x)).reset_index().rename(columns={'DESCRIPTION': 'MEDICATIONS'})
immunizations_grouped = immunizations_covid_patients.groupby('PATIENT')['DESCRIPTION'].apply(lambda x: ', '.join(x)).reset_index().rename(columns={'DESCRIPTION': 'IMMUNIZATIONS'})

final_symptoms = final_symptoms.merge(medications_grouped, on='PATIENT', how='left')
final_symptoms = final_symptoms.merge(immunizations_grouped, on='PATIENT', how='left')

# Add encounter type (hospital/ER/ambulatory) for COVID
covid_encounters = encounters_df[encounters_df['REASONDESCRIPTION'].str.contains('COVID', case=False, na=False)]
encounter_types = covid_encounters.groupby('PATIENT')['ENCOUNTERCLASS'].apply(lambda x: ', '.join(x)).reset_index().rename(columns={'ENCOUNTERCLASS': 'ENCOUNTER_TYPE'})

final_symptoms = final_symptoms.merge(encounter_types, on='PATIENT', how='left')

# Add hospitalization yes/no
final_symptoms['HOSPITALIZED'] = final_symptoms['ENCOUNTER_TYPE'].apply(lambda x: 'Yes' if pd.notna(x) and ('inpatient' in x.lower() or 'emergency' in x.lower()) else 'No')

# Add vaccination timing (before or after COVID recovery)
immunizations_df['DATE'] = pd.to_datetime(immunizations_df['DATE'])
earliest_vaccination = immunizations_df.groupby('PATIENT')['DATE'].min().reset_index().rename(columns={'DATE': 'FIRST_VACCINATION_DATE'})

final_symptoms = final_symptoms.merge(earliest_vaccination, on='PATIENT', how='left')

final_symptoms['VACCINATION_BEFORE_COVID'] = final_symptoms.apply(
    lambda row: 'Yes' if pd.notna(row['FIRST_VACCINATION_DATE']) and row['FIRST_VACCINATION_DATE'] < row['RECOVERY_DATE'] else 'No', axis=1
)

# Fill missing text fields
final_symptoms['MEDICATIONS'] = final_symptoms['MEDICATIONS'].fillna('None')
final_symptoms['IMMUNIZATIONS'] = final_symptoms['IMMUNIZATIONS'].fillna('None')
final_symptoms['COMORBIDITIES'] = final_symptoms['COMORBIDITIES'].fillna('None')

final_symptoms.head(10)


In [20]:
# Dropping unnecessary columns
reduced_final_symptoms = final_symptoms.drop(columns=[
    'PATIENT', 'DATE', 'RECOVERY_DATE', 'ETHNICITY', 'RACE', 'ENCOUNTER_TYPE', 'HOSPITALIZED', 'FIRST_VACCINATION_DATE'
])

In [21]:
# Define clean-up function
def clean_text(text):
    if isinstance(text, str):
        text = text.lower()                 # Lowercase
        text = text.strip()                  # Remove leading/trailing spaces
        text = re.sub(r'\s+', ' ', text)      # Replace multiple spaces with single space
    return text

# Apply to text columns
text_columns = ['DESCRIPTION', 'VALUE', 'UNITS', 'post_covid_symptom', 'GENDER', 'COMORBIDITIES', 'MEDICATIONS', 'IMMUNIZATIONS', 'VACCINATION_BEFORE_COVID']

for col in text_columns:
    reduced_final_symptoms[col] = reduced_final_symptoms[col].apply(clean_text)


In [22]:
reduced_final_symptoms.head()

,DESCRIPTION,days_after_recovery,VALUE,UNITS,post_covid_symptom,GENDER,AGE,COMORBIDITIES,MEDICATIONS,IMMUNIZATIONS,VACCINATION_BEFORE_COVID
0,pain severity - 0-10 verbal numeric rating [sc...,91,3.0,{score},unlikely related,f,5,"dyspnea (finding), wheezing (finding), fever (...",0.4 ml enoxaparin sodium 100 mg/ml prefilled s...,"hep b adolescent or pediatric, hep b adolescen...",yes
1,pain severity - 0-10 verbal numeric rating [sc...,82,3.0,{score},unlikely related,f,5,"dyspnea (finding), wheezing (finding), fever (...",0.4 ml enoxaparin sodium 100 mg/ml prefilled s...,"hep b adolescent or pediatric, hep b adolescen...",yes
2,pain severity - 0-10 verbal numeric rating [sc...,74,3.0,{score},unlikely related,f,54,"perennial allergic rhinitis, cardiac arrest, h...",diphenhydramine hydrochloride 25 mg oral table...,influenza seasonal injectable preservative free,no
3,pain severity - 0-10 verbal numeric rating [sc...,55,3.0,{score},likely related,f,54,"perennial allergic rhinitis, cardiac arrest, h...",diphenhydramine hydrochloride 25 mg oral table...,influenza seasonal injectable preservative free,no
4,pain severity - 0-10 verbal numeric rating [sc...,6,2.0,{score},likely related,f,66,"hypertension, chronic sinusitis (disorder), ca...","hydrochlorothiazide 25 mg oral tablet, hydroch...",influenza seasonal injectable preservative fre...,no


## Data split normal

In [23]:
# Converting final_symptoms DataFrame to a Hugging Face Dataset
hf_dataset = Dataset.from_pandas(reduced_final_symptoms)

# Split into 80/20
dataset_split = hf_dataset.train_test_split(test_size=0.2, shuffle=True, seed=42)

# Create output directory
output_dir = os.path.join("./", "80-20-normal-data")
os.makedirs(output_dir, exist_ok=True)

# Save each part as CSV
dataset_split['train'].to_csv(os.path.join(output_dir, "normal-train.csv"))
dataset_split['test'].to_csv(os.path.join(output_dir, "normal-test.csv"))

print(f"Train and test datasets saved successfully to {output_dir}")


Creating CSV from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Train and test datasets saved successfully to ./80-20-normal-data


## Data preparation for llm usages

In [25]:
llm_train_df = pd.read_csv(os.path.join(output_dir, "normal-train.csv"))
llm_test_df = pd.read_csv(os.path.join(output_dir, "normal-test.csv"))

In [26]:
def create_llm_training_narrative(row):
    narrative = (
        f"A {row['AGE']}-year-old {row['GENDER']} patient "
        f"experienced the symptom '{row['DESCRIPTION']}' "
        f"{row['days_after_recovery']} days after COVID-19 recovery. "
        f"The symptom was measured as {row['VALUE']} {row['UNITS']}. "
        f"Comorbidities include: {row['COMORBIDITIES']}. "
        f"Medications received: {row['MEDICATIONS']}. "
        f"Immunizations: {row['IMMUNIZATIONS']}. "
        f"Vaccination before COVID recovery: {row['VACCINATION_BEFORE_COVID']}."
    )
    return narrative

In [27]:
# Applying narrative function
llm_train_df['llm_patient_narrative'] = llm_train_df.apply(create_llm_training_narrative, axis=1)
llm_test_df['llm_patient_narrative'] = llm_test_df.apply(create_llm_training_narrative, axis=1)

# Keeping only 'llm_patient_narrative' and 'post_covid_symptom' (no renaming!)
llm_training_data = llm_train_df[['llm_patient_narrative', 'post_covid_symptom']].rename(columns={'post_covid_symptom': 'label'})

# Select and rename columns for testing data
llm_test_data = llm_test_df[['llm_patient_narrative', 'post_covid_symptom']].rename(columns={'post_covid_symptom': 'label'})


In [28]:
# 80-20-llm-data directory
llm_output_dir = os.path.join("./", "80-20-llm-data")
os.makedirs(llm_output_dir, exist_ok=True)

llm_training_data.to_csv(os.path.join(llm_output_dir, "llm_train.csv"), index=False)
llm_test_data.to_csv(os.path.join(llm_output_dir, "llm_test.csv"), index=False)

print(f"LLM train and test datasets saved successfully at {llm_output_dir}")

LLM train and test datasets saved successfully at ./80-20-llm-data
